In [48]:
%load_ext autoreload
%autoreload 2
from utils import SMILESVocab

from utils import vae_loss
from vae import GraphEncoder, SMILESDecoder, VAE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
test_smi = "CCO"
tokens = SMILESVocab.tokenize_smiles(test_smi)
print("分词结果:", tokens)  # 应输出 ['C', 'C', 'O']



分词结果: ['C', 'C', 'O']


In [1]:
%load_ext autoreload
%autoreload 2
from utils import build_vocab_large, preprocess_to_h5
data_path = '/Users/bmw/Research/data/data/pubchem-10m-clean.txt'
h5_path ='/Users/bmw/Research/data/data/pubchem.h5'


In [2]:
vocab = build_vocab_large(data_path)


Building Vocab: 9999918it [00:50, 199891.60it/s]


In [1]:
from rdkit import Chem

In [2]:
smi = '[O][C][Branch1][C][C][C]'
mol = Chem.MolFromSmiles(smi)

[17:17:20] SMILES Parse Error: syntax error while parsing: [O][C][Branch1][C][C][C]
[17:17:20] SMILES Parse Error: Failed parsing SMILES '[O][C][Branch1][C][C][C]' for input: '[O][C][Branch1][C][C][C]'


In [41]:
preprocess_to_h5(data_path, vocab, 'pubchem.h5')

Preprocessing: 442942it [14:28, 523.55it/s][12:50:03] Can't kekulize mol.  Unkekulized atoms: 14 15 16 17 18 27 28 29 30 31 32 33 34 35 36 37 38 39 40 49 50 51 52 53 54 55 56 57 66 67 68 69 70 71 72 73 74 83 84 85 86 87 88 89 90 91 100 101 102 103 104 105 106 107 108 109
Preprocessing: 443048it [14:28, 521.85it/s]

Skipping due to error: wrong SMILES: CCCCCCCCC1=CC=C2C(=C3c4c5ccc(CCCCCCCC)ccc5c5c4c4c6c7c8c(c9ccc(CCCCCCCC)ccc9c8c8c9ccc(CCCCCCCC)ccc9c9c%10c%11ccc(CCCCCCCC)ccc%11c%11c%12c%13ccc(CCCCCCCC)ccc%13c5c%12c4c(c7c89)c%11%10)C2C36)C=C1


Preprocessing: 6235190it [3:19:29, 536.10it/s][15:55:04] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61
Preprocessing: 6235301it [3:19:29, 540.46it/s]

Skipping due to error: wrong SMILES: C1=CC=C2C(=C3c4c5cccccc5c5c4c4c6c7c8c(c9cccccc9c8c8c9cccccc9c9c%10c%11cccccc%11c%11c%12c%13cccccc%13c5c%12c4c(c7c89)c%11%10)C2C36)C=C1


Preprocessing: 9999918it [5:16:45, 526.16it/s]

Total skipped molecules: 2


In [42]:
import h5py
h5 = h5py.File(h5_path, "r")

In [43]:

with h5py.File(h5_path, "r") as h5:
    print("Available keys:", list(h5.keys()))

Available keys: ['data']


In [44]:
with h5py.File(h5_path, "r") as h5:
    data_group = h5["data"]
    print("Keys inside /data:", list(data_group.keys()))

Keys inside /data: ['edge_attr', 'edge_index', 'fgp', 'seq', 'x']


In [45]:
with h5py.File(h5_path, "r") as h5:
    x_data = h5["data/x"][:]  # Load entire dataset
    print("Shape of x:", x_data.shape)

KeyboardInterrupt: 

In [17]:
import torch
from tqdm import tqdm
import pandas as pd
DEVICE = torch.device("cpu")
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")

In [12]:
state_dict_path = './models/ckpt/CLR_model_MolCLR.pth'
CLRmodel = torch.load(state_dict_path, map_location=DEVICE, weights_only=False)
graphmodel = CLRmodel.graphmodel

In [15]:
encoder = GraphEncoder(graphmodel)
decoder = SMILESDecoder(vocab=vocab)
vae = VAE(encoder=encoder, decoder=decoder).to(DEVICE)

In [46]:
%load_ext autoreload
%autoreload 2
from data import MoleculeLoaderWrapper
train_loader, val_loader, test_loader = MoleculeLoaderWrapper(batch_size=16, num_workers=0, valid_size=0.1,
                                                              test_size=0.1, h5_path=h5_path, vocab = vocab,
                                                               splitting='random' ).get_data_loaders()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Dataset is random splitting...


/opt/miniconda3/envs/torch/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [49]:
for data, seq, fgp in tqdm(train_loader):
    print(data)
    data = data.to(DEVICE)
    seq = seq.to(DEVICE)

    recon_logits, mu, logvar = vae(data, seq[:, :-1])
    loss = vae_loss(recon_logits, seq[:, 1:], mu=mu, log_var=logvar, beta=0.5)
    print(loss)




  0%|          | 1/499996 [00:08<1175:38:41,  8.46s/it]

DataBatch(x=[434, 2], edge_index=[2, 930], edge_attr=[930, 2], batch=[434], ptr=[17])
tensor(233.7912, grad_fn=<AddBackward0>)


  0%|          | 2/499996 [00:08<500:34:59,  3.60s/it] 

DataBatch(x=[395, 2], edge_index=[2, 844], edge_attr=[844, 2], batch=[395], ptr=[17])
tensor(239.2679, grad_fn=<AddBackward0>)


  0%|          | 3/499996 [00:08<284:36:55,  2.05s/it]

...

tensor(211.3703, grad_fn=<AddBackward0>)
DataBatch(x=[380, 2], edge_index=[2, 806], edge_attr=[806, 2], batch=[380], ptr=[17])
tensor(225.7902, grad_fn=<AddBackward0>)


  0%|          | 57/499996 [00:18<24:26:48,  5.68it/s]

DataBatch(x=[426, 2], edge_index=[2, 906], edge_attr=[906, 2], batch=[426], ptr=[17])
tensor(238.6605, grad_fn=<AddBackward0>)
DataBatch(x=[326, 2], edge_index=[2, 682], edge_attr=[682, 2], batch=[326], ptr=[17])


  0%|          | 57/499996 [00:20<49:14:13,  2.82it/s]


KeyboardInterrupt: 

In [5]:
vocab.print_summary()

Vocab Size: 12
Special Tokens:
  <bos>: 8
  <eos>: 9
  <pad>: 10
  <unk>: 11

First 20 Chemical Tokens:
  #: 0
  1: 1
  Br: 2
  C: 3
  Cl: 4
  N: 5
  O: 6
  c: 7


In [6]:
test_smiles = "ClC(Br)N=O"
encoded = vocab.encode(test_smiles, add_bos=True, add_eos=True)
decoded = vocab.decode(encoded)
print(f"Original: {test_smiles}")
print(f"Decoded: {decoded}")

Original: ClC(Br)N=O
Decoded: ClC<unk>Br<unk>N<unk>O


In [7]:
print(encoded)

[8, 4, 3, 11, 2, 11, 5, 11, 6, 9]


In [8]:
vocab.pad_idx

10

In [1]:
from utils import vae_loss, load_vocab_from_pickle, KLAnnealer

In [3]:
vocab = load_vocab_from_pickle('/Users/bmw/Research/data/vae/pubchem_vocab.pkl')

In [4]:
vocab.print_summary()

Vocab Size: 341
Special Tokens:
  <bos>: 337
  <eos>: 338
  <pad>: 339
  <unk>: 340

First 20 Chemical Tokens:
  #: 0
  %10: 1
  %11: 2
  %12: 3
  %13: 4
  %14: 5
  %15: 6
  %16: 7
  %17: 8
  %18: 9
  %19: 10
  %20: 11
  %21: 12
  %22: 13
  %23: 14
  %24: 15
  %25: 16
  %26: 17
  %27: 18
  %28: 19


In [11]:
import pandas as pd
df = pd.read_csv('/Users/bmw/Research/data/data/data.csv')
df.head(-1)

,SMILES,TYPES,CO2,O2,N2,CO2/O2,CO2/N2,SELFIES
0,Nc1ccccc1,Conjugated Polymers,0.493917,0.119179,0.012975,4.144323,38.06543,[N][C][=C][C][=C][C][=C][Ring1][=Branch1]
1,CONc1ccccc1,Conjugated Polymers,1.060000,0.201905,0.047007,5.250000,22.55000,[C][O][N][C][=C][C][=C][C][=C][Ring1][=Branch1]
2,CCONc1ccccc1,Conjugated Polymers,2.220000,0.620112,0.106988,3.580000,20.75000,[C][C][O][N][C][=C][C][=C][C][=C][Ring1][=Bran...
3,Cn1cccc1,Conjugated Polymers,4.860000,1.650174,0.244970,2.945143,19.83915,[C][N][C][=C][C][=C][Ring1][Branch1]
4,CCCCCCCCCCCCc1ccsc1,Conjugated Polymers,88.200000,20.183070,9.402985,4.370000,9.38000,[C][C][C][C][C][C][C][C][C][C][C][C][C][C][=C]...
...,...,...,...,...,...,...,...,...
590,OCCCC,Thermally Rearranged Polymers,163.333333,20.993300,8.300095,7.780261,19.67849,[O][C][C][C][C]
591,OCC,Thermally Rearranged Polymers,69.000000,7.995365,2.400000,8.630000,28.75000,[O][C][C]
592,C[SiH](C1=CC=C([Si](C)(CC[Si](C)(CC)C)C)C=C1)C,Various Polymers,4.010000,1.110803,0.257051,3.610000,15.60000,[C][SiH1][Branch2][Ring1][=C][C][=C][C][=C][Br...
593,OCC,Various Polymers,53.500000,2.690265,0.924933,19.886520,57.84205,[O][C][C]


In [9]:
results = []
for smi in df['SMILES']:
    encoded = vocab.encode(test_smi, add_bos=True, add_eos=True)
    decoded = vocab.decode(encoded)
    # print(f"Original: {test_smi}")
    # print(f"Decoded: {decoded}")
    results.append(decoded == test_smi)

In [10]:
sum(results)

596